In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from IPython.display import Image
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

import selenium
from selenium import webdriver
import csv
import time
import urllib
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,ElementNotInteractableException,ElementNotVisibleException,TimeoutException,ElementClickInterceptedException

In [81]:
def scrap_carsdekho(web_link):
    driver=webdriver.Chrome(r"C:\Users\User\Documents\chromedriver") ##Opening web driver
    driver.get(web_link) ## Opeing web link
    time.sleep(1)
    location_selection_link=driver.find_element_by_xpath("//li[@data-slug='/usedCars']/a").get_attribute('href')
    driver.get(location_selection_link)
    time.sleep(1)
    car_links=[]
    for number,location in enumerate(driver.find_elements_by_xpath("//ul[@class='listing gsc_col-md-9 gsc_col-lg-9']/li")):
#         print(number)
        time.sleep(2)
        driver.find_element_by_xpath(f"//ul[@class='listing gsc_col-md-9 gsc_col-lg-9']/li[{number+1}]").click()
        n=0
        i=900
        while n!=10:
            driver.execute_script("window.scrollBy(0, arguments[0]);", i)
            time.sleep(5)
            n+=1
            i+=250
        for link in driver.find_elements_by_xpath("//div[@class='gsc_col-xs-7 carsName']"):
            car_links.append(link.find_element_by_xpath('a').get_attribute('href')) # each car link
        driver.get(location_selection_link)
    print(len(car_links))
    values={
         'Car_name':[],
         'Price' : [],
         'Kilometers': [],
         'Year of Purchase': [],
         'Owner': [],
         'Fuel': [],
         'Transmission': [],
         'RTO': [],
#          'Insurance': [],
         'Insurance Type': [],
         'Feature' : [],
         'Safety' : [],
         'Interior' : [],
         'Exterior' : []
           }
    
    for link in car_links:
        try:
            driver.get(link)
            engin_ext_int=[]
            values['Car_name'].append(driver.find_element_by_xpath("//div[@class='gsc_col-xs-12']/h1").text)
            values['Price'].append(driver.find_element_by_xpath("//div[@class='priceSection']").text)
            values["Insurance Type"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[9]").text.split('\n')[1])
            values["RTO"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[7]").text.split('\n')[1])
            values["Transmission"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[8]").text.split('\n')[1])
            values["Fuel"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[3]").text.split('\n')[1])
            values["Owner"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[6]").text.split('\n')[1])
            values["Year of Purchase"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[1]").text.split('\n')[1])
            values["Kilometers"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[4]").text.split('\n')[1])
            
            plus_buttons=driver.find_elements_by_css_selector("div.content  ")
            for button in plus_buttons:
                driver.execute_script("arguments[0].setAttribute('class','content  on')", button) ## Opening all plus buttons
            try:
                driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[1]/div")
                values['Feature'].append(', '.join(driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[1]/div").text.split("\n")))
            except NoSuchElementException as e:
                values['Feature'].append("-")
            try:
                driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[2]/div")
                values['Safety'].append(', '.join(driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[2]/div").text.split("\n")))
            except NoSuchElementException as e:
                values['Safety'].append('-')
            try:
                driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[3]/div")
                values['Interior'].append(', '.join(driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[3]/div").text.split("\n")))
            except NoSuchElementException as e:
                values['Interior'].append('-')
            try:
                driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[4]/div")
                values['Exterior'].append(', '.join(driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[4]/div").text.split("\n")))
            except NoSuchElementException as e:
                values['Exterior'].append('-')
        except NoSuchElementException as e:
            continue
    return values
data=scrap_carsdekho('https://www.cardekho.com/')
pd.DataFrame(data).to_csv("cardekho.csv")
pd.DataFrame(data)

1585


,Car_name,Price,Kilometers,Year of Purchase,Owner,Fuel,Transmission,RTO,Insurance Type,Feature,Safety,Interior,Exterior
0,2011 Maruti Ritz,"₹ 1,98,410","54,004 Kms",2010,2nd Owner,Petrol,Manual,GJ01,Third Party insurance,"Foldable Rear Seat, Power Steering, Power Wind...","Centeral Locking, Power Door Locks, Child Safe...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ..."
1,2014 Hyundai Verna,"₹ 4,85,492","65,484 Kms",2014,2nd Owner,Diesel,Manual,GJ01,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."
2,2018 Maruti Wagon R,"₹ 4,41,025","23,197 Kms",2018,1st Owner,Petrol,Manual,GJ18,Third Party insurance,"Foldable Rear Seat, Power Steering, Power Wind...","Centeral Locking, Power Door Locks, Child Safe...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."
3,2017 Maruti Wagon R,"₹ 4,05,188","43,031 Kms",2017,1st Owner,Petrol,Manual,GJ18,Third Party insurance,"Foldable Rear Seat, Power Steering, Power Wind...","Centeral Locking, Power Door Locks, Child Safe...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."
4,2019 Maruti S-Presso,"₹ 4,15,500","24,230 Kms",2019,1st Owner,Petrol,Manual,GJ01,Third Party insurance,"Power Steering, Power Windows Front, Low Fuel ...","Anti Lock Braking System, Centeral Locking, Ch...","Air Conditioner, Heater, Digital Odometer, Ele...","Adjustable Head Lights, Manually Adjustable Ex..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,2018 Hyundai i20,"₹ 5,65,000","5,663 Kms",2018,1st Owner,Petrol,Manual,MH12,Third Party insurance,"Power Steering, Power Windows Front, Low Fuel ...","Anti Lock Braking System, Centeral Locking, Ch...","Air Conditioner, Heater, Digital Odometer, Tac...","Adjustable Head Lights, Manually Adjustable Ex..."
827,2018 Maruti Alto 800,"₹ 3,75,000","29,433 Kms",2018,1st Owner,Petrol,Manual,MH12,Third Party insurance,"Power Steering, Power Windows Front, Remote Tr...","Centeral Locking, Child Safety Locks, Passenge...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Manually Adjustable Ex..."
828,2015 Honda City,"₹ 7,80,056","43,155 Kms",2015,1st Owner,Petrol,Automatic,MH12,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."
829,2018 Maruti Ciaz,"₹ 8,20,000","10,265 Kms",2018,1st Owner,Petrol,Manual,MH12,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."


In [76]:
for i in data.keys():
    print(i,len(data[i]))

Car_name 757
Price 757
Kilometers 757
Year of Purchase 757
Owner 757
Fuel 757
Transmission 757
RTO 757
Insurance Type 757
Feature 756
Safety 756
Interior 756
Exterior 754


In [77]:
data1=data.copy()

In [2]:
driver=webdriver.Chrome(r"C:\Users\User\Documents\chromedriver") ##Opening web driver

In [11]:
driver.get('https://www.cardekho.com/used-car-details/used-MG-Hector-2019-2021-Hybrid-Smart-MT-BSIV-cars-Ahmedabad_28CCAB984C731192F0602CB1F8819955.htm')

In [4]:
driver.find_element_by_xpath("//div[@class='vdp-head']").text # car Name

'2019MG Hector Hybrid Smart MT BSIV'

In [5]:
driver.find_element_by_xpath("//span[@class='amount']").text  # Price

'14.9 Lakh*'

In [6]:
driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[1]").text # Year
driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[3]").text # Kms
driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[4]").text # Owner
driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[5]").text # Fuel
driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[6]").text # Transmission
driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[7]").text # Comprehensive
driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[8]").text # RTO

'GJ01'

In [16]:
## Specification 
driver.find_element_by_xpath("//a[@class=' viewMorebtn viewMorebtnArrow']").click() ## view More
for key,val in zip(driver.find_element_by_xpath("//div[@class='SpecsFeatureList']").text.split('\n')[0::2],driver.find_element_by_xpath("//div[@class='SpecsFeatureList']").text.split('\n')[1::2]): ## Specification all data
    l.append(key)

In [18]:
print(l)

['Mileage', 'Engine', 'Max Power', 'Torque', 'Seats', 'Color', 'Gear Box', 'Drive Type', 'Seating Capacity', 'Front Brake Type', 'Rear Brake Type', 'Tyre Type', 'Alloy Wheel Size', 'No Of Doors', 'Cargo Volume', 'Displacement', 'Max Power', 'Max Torque', 'No Of Cylinder', 'Valves Per Cylinder', 'Turbo Charger', 'Length', 'Width', 'Height', 'Wheel Base']


In [8]:
## Features
driver.find_element_by_xpath("//ul[@class='gsc-ta-clickWrap ']/li[2]").click()

# under feature
driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[1]").click() #car confort features
', '.join(driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[1]/div").text.replace('• ','').split('\n'))

driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[2]").click() #car Safety features
', '.join(driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[2]/div").text.replace('• ','').split('\n'))

driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[3]").click() #car Entertainement features
', '.join(driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[3]/div").text.replace('• ','').split('\n'))

driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[4]").click() #car Exterior features
', '.join(driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[4]/div").text.replace('• ','').split('\n'))

In [28]:
def scrap_carsdekho(web_link):
    driver=webdriver.Chrome(r"C:\Users\User\Documents\chromedriver") ##Opening web driver
    driver.get(web_link) ## Opeing web link
    time.sleep(1)
    location_selection_link=driver.find_element_by_xpath("//li[@data-slug='/usedCars']/a").get_attribute('href')
    driver.get(location_selection_link)
    time.sleep(1)
    car_links=[]
    for number,location in enumerate(driver.find_elements_by_xpath("//ul[@class='listing gsc_col-md-9 gsc_col-lg-9']/li")):
#         print(number)
        time.sleep(2)
        driver.find_element_by_xpath(f"//ul[@class='listing gsc_col-md-9 gsc_col-lg-9']/li[{number+1}]").click()
        n=0
        i=900
        while n!=10:
            driver.execute_script("window.scrollBy(0, arguments[0]);", i)
            time.sleep(5)
            n+=1
            i+=250
        for link in driver.find_elements_by_xpath("//div[@class='gsc_col-xs-7 carsName']"):
            car_links.append(link.find_element_by_xpath('a').get_attribute('href')) # each car link
        driver.get(location_selection_link)
    print(len(car_links))
    values={
         'Car_name':[],
         'Price' : [],
         'Kilometers': [],
         'Year of Purchase': [],
         'Owner': [],
         'Fuel': [],
         'Transmission': [],
         'RTO': [],
#          'Insurance': [],
#          'Wheel Size': [],
         'Specifications':[],
         'Confort Feature' : [],
         'Safety' : [],
         'Entertainment Feature' : [],
         'Exterior' : [],
        
    }
    specification_keys=['Mileage', 'Engine', 'Max Power', 'Torque', 'Seats', 'Color', 'Gear Box']
    for key in specificat
    ion_keys:
        values[key]=[]
    
    
    for link in car_links:
        try:
            driver.get(link)
            engin_ext_int=[]
            values['Car_name'].append(driver.find_element_by_xpath("//div[@class='vdp-head']").text)
            values['Price'].append(driver.find_element_by_xpath("//span[@class='amount']").text)
            values["RTO"].append(driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[8]").text)
            values["Transmission"].append(driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[6]").text)
            values["Fuel"].append(driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[5]").text)
            values["Owner"].append(driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[4]").text)
            values["Year of Purchase"].append(driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[1]").text)
            values["Kilometers"].append(driver.find_element_by_xpath("//ul[@class='gsc_row clearfix']/li[3]").text)
                                                                
                    
            ## Specification 
#             driver.find_element_by_xpath("//a[@class=' viewMorebtn viewMorebtnArrow']").click() ## view More
            if 'Wheel Size' not in driver.find_element_by_xpath("//div[@class='SpecsFeatureList']").text.split('\n')[0::2]:
                values['Wheel Size'].append('-')
            for key,val in zip(driver.find_element_by_xpath("//div[@class='SpecsFeatureList']").text.split('\n')[0::2],driver.find_element_by_xpath("//div[@class='SpecsFeatureList']").text.split('\n')[1::2]): ## Specification all data                
                values[key].append(val)
            
                                                                
            ## Features
            driver.find_element_by_xpath("//ul[@class='gsc-ta-clickWrap ']/li[2]").click()
            
            driver.find_element_by_xpath("a[class=' viewMorebtn viewMorebtnArrow']").click()  ## View More

            # under feature
            driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[1]").click() #car confort features
            values["Confort Feature"].append(', '.join(driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[1]/div").text.replace('• ','').split('\n')))

            driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[2]").click() #car Safety features
            values["Safety"].append(', '.join(driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[2]/div").text.replace('• ','').split('\n')))

            driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[3]").click() #car Entertainement features
            values["Entertainment Feature"].append(', '.join(driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[3]/div").text.replace('• ','').split('\n')))

            driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[4]").click() #car Exterior features
            values["Exterior"].append(', '.join(driver.find_element_by_xpath("//div[@class='reserachfilter marginTop20']/div[4]/div").text.replace('• ','').split('\n')))


        except NoSuchElementException as e:
            continue
    return values
data=scrap_carsdekho('https://www.cardekho.com/')
pd.DataFrame(data).to_csv("cardekho11.csv")
pd.DataFrame(data)

1633


KeyError: 'Drive Type'